In [ ]:
import os
import sys
import datetime

import pickle
from keras import mixed_precision
from sklearn.model_selection import train_test_split

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../")))
from SRModels.deep_learning_models.EDSR_model import EDSR
from SRModels.loading_methods import load_dataset_as_patches
from SRModels.constants import EDSR_PATCH_SIZE, EDSR_STRIDE, RANDOM_SEED, EDSR_SCALE_FACTOR

mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060 Ti, compute capability 8.6


In [4]:
HR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/HR"))
LR_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../data/images/LR"))

In [ ]:
X, Y = load_dataset_as_patches(HR_ROOT, LR_ROOT, mode="scale", patch_size=EDSR_PATCH_SIZE, stride=EDSR_STRIDE, scale_factor=EDSR_SCALE_FACTOR)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, shuffle=True, random_state=RANDOM_SEED)

print(f"X_train shape: {X_train.shape}, Y_train shape: {Y_train.shape}")
print(f"X_val shape: {X_val.shape}, Y_val shape: {Y_val.shape}")
print(f"X_test shape: {X_test.shape}, Y_test shape: {Y_test.shape}")

X_train shape: (7873, 24, 24, 3), Y_train shape: (7873, 48, 48, 3)
X_val shape: (875, 24, 24, 3), Y_val shape: (875, 48, 48, 3)
X_test shape: (972, 24, 24, 3), Y_test shape: (972, 48, 48, 3)


In [ ]:
model = EDSR()

model.setup_model(scale_factor=EDSR_SCALE_FACTOR, num_res_blocks=16, num_filters=64)

Model: "EDSR"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   1792        ['input[0][0]']                  
                                64)                                                               
                                                                                                  
 separable_conv2d (SeparableCon  (None, None, None,   4736       ['conv2d[0][0]']                 
 v2D)                           64)                                                            

In [ ]:
# Train EDSR and capture callbacks for metrics
time_cb, mem_cb = model.fit(
    X_train, Y_train, X_val, Y_val, 
    batch_size=16, 
    epochs=100, 
    use_augmentation=True, 
    use_mix=True
)

In [ ]:
# Evaluate and prepare metrics dictionary
results = model.evaluate(X_test, Y_test)
metrics_dict = {
    "eval_loss": float(results[0]),
    "eval_psnr": float(results[1]),
    "eval_ssim": float(results[2]),
    "epoch_time_sec": time_cb.mean_time_value(),
    "memory": mem_cb.as_dict()
}
print(metrics_dict)

857/857 [==============================] - 15s 18ms/step - loss: 0.0288 - psnr: 29.4717 - ssim: 0.7602
Loss: 0.0288, PSNR: 29.47 dB, SSIM: 0.7602


[0.028815768659114838, 29.47173500061035, 0.7601730227470398]

In [ ]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

model.save(directory=f"models/EDSR/EDSR_{timestamp}", timestamp=timestamp)

Model saved to models/EDSR\EDSR_x2_20250626_183609.h5


In [ ]:
# Save evaluation/time/memory metrics next to the model
metrics_path = os.path.abspath(os.path.join(os.getcwd(), f"models/EDSR/EDSR_{timestamp}/EDSR_{timestamp}_metrics.pkl"))

with open(metrics_path, "wb") as f:
    pickle.dump(metrics_dict, f)
    
print(f"Saved metrics to {metrics_path}")